## Notebook to read in the data from a few papers that compile the observed NS masses and kicks



In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import ascii
from astropy.table import Table
import re


### define some latex cleanup functions

In [3]:
# Make a latex clean up function
def clean_column_name(name):
    # Remove LaTeX formatting
    # name = re.sub(r'\\[a-zA-Z]+', '', name)  # Remove LaTeX commands
    # name = re.sub(r'\{|\}', '', name)  # Remove curly braces
    name = re.sub(r'\$', '', name)  # Remove $ signs
    name = re.sub(r'\\rm', '', name)  # Remove $ signs
    name = re.sub(r'\\', '', name)  # Remove $ signs
    return name.strip()


def clean_latex_table(table):
    table = table.copy()

    # Clean the column names in You24_DataTable1
    table.rename(columns=lambda x: clean_column_name(x), inplace=True)
    
    # Check for \pm values and create extra columns if needed
    pattern = re.compile(r'\^{\+([^}]*)}\_{-([^}]*)}')
    for key in table.keys():
        if any(table[key].apply(lambda x: bool(pattern.search(str(x))))):
            table[key + '_up'] = np.nan
            table[key + '_down'] = None

    for key in table.keys():
        for i in range(len(table[key])):
            # check if the data is a string
            if not isinstance(table.loc[i,key], str):
                continue
            # First remove all the $ signs
            table.loc[i,key] = table.loc[i,key].replace('$','')

            # Search for the pattern ^{+*}_{-*}
            match = re.search(r'\^{\+([^}]*)}\_{-([^}]*)}', table.loc[i, key])
            if match:
                up_value = match.group(1)
                down_value = match.group(2)
                table.loc[i, key+'_up'] = up_value
                table.loc[i, key+'_down'] = down_value
                table.loc[i, key] = re.sub(r'\^{\+[^}]*}\_{-[^}]*}', '', table.loc[i, key])

    return table

## Starting with [You et al. 2024](http://arxiv.org/abs/2412.05524) 
(the Nature paper) Extended Data Table 1 and 2

In [5]:
You24_DataTable1 = ascii.read('../data/You_NSmass_2024_DataTable1.tex', format='latex', header_start=7, data_start=8).to_pandas()
print(len(You24_DataTable1))

You24_DataTable2 = ascii.read('../data/You_NSmass_2024_DataTable2.tex', format='latex', header_start=6, data_start=7).to_pandas()
print(len(You24_DataTable2))

# Combine the tables
You24_DataTable = pd.concat([You24_DataTable1, You24_DataTable2], axis=0, ignore_index=True)
# display(You24_DataTable)

You24_DataTable_clean = clean_latex_table(You24_DataTable) 
display(You24_DataTable_clean)

40
35


/var/folders/mc/hhcvbs7n2jq7d_hr2d6qvjhm0000gq/T/ipykernel_3951/2331560644.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.12' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  table.loc[i, key+'_up'] = up_value
/var/folders/mc/hhcvbs7n2jq7d_hr2d6qvjhm0000gq/T/ipykernel_3951/2331560644.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.11' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  table.loc[i, key+'_up'] = up_value


,Name,m_{ r} (mathrm{M}_{odot}),m_{ s} (mathrm{M}_{odot}),m_{ c} (mathrm{M}_{odot}),P (ms),"dot{P},(10^{-18})",P_{ b} (day),e_0,Reference,m_{ r} (mathrm{M}_{odot})_up,m_{ r} (mathrm{M}_{odot})_down,m_{ s} (mathrm{M}_{odot})_up,m_{ s} (mathrm{M}_{odot})_down
0,J0514-4002A,1.25(6),1.22(6),--,4.99,0.0840,18.785,0.888,\cite{Ridolfi19_J0514a},NaN,None,NaN,None
1,J1829+2456,1.306(4),1.299(4),--,41.01,0.0435,1.176,0.139,\cite{PSRJ1829mass},NaN,None,NaN,None
2,J1906+0746,1.322(11),1.291(11),--,144.10,20268.0000,0.166,0.085,\cite{PSR1906},NaN,None,NaN,None
3,B1534+12,1.3330(2),1.3455(2),--,37.90,2.3640,0.421,0.274,\cite{FonsecaB1534},NaN,None,NaN,None
4,J0737-3039A,1.3381(7),1.2489(7),--,22.70,1.7610,0.102,0.088,\cite{Kramer06Sci},NaN,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,GW170817,1.34,1.38,--,NaN,NaN,NaN,NaN,"\cite{gw170817,Zhu_GW170817}",0.12,0.09,0.11,0.11
71,GW190425,1.64,1.66,--,NaN,NaN,NaN,NaN,"\cite{GW190425,Zhu_GW170817}",0.13,0.11,0.12,0.12
72,GW191219,--,1.17,31.1,NaN,NaN,NaN,NaN,\cite{Abbott_GW191219_GWTC3},NaN,None,0.07,0.06
73,GW200115,--,1.4,5.9,NaN,NaN,NaN,NaN,\cite{Abbott_GW200115},NaN,None,0.6,0.2


## also the data from [Zhao et al 2023](https://ui.adsabs.harvard.edu/abs/2023MNRAS.525.1498Z)
Their table 3

In [6]:
Zhao_2023_Table3a = ascii.read('../data/Zhao_2023_Table3a.tex', format='latex', header_start=6, data_start=9).to_pandas()
# display(Zhao_2023_Table3a)
Zhao_2023_Table3b = ascii.read('../data/Zhao_2023_Table3b.tex', format='latex', header_start=6, data_start=8).to_pandas()
# display(Zhao_2023_Table3b)

# Combine the tables
Zhao_2023_Table3 = pd.concat([Zhao_2023_Table3a, Zhao_2023_Table3b], axis=0, ignore_index=True)
# display(Zhao_2023_Table3)

Zhao_2023_Table3_clean = clean_latex_table(Zhao_2023_Table3) 
display(Zhao_2023_Table3_clean)

/var/folders/mc/hhcvbs7n2jq7d_hr2d6qvjhm0000gq/T/ipykernel_3951/2331560644.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  table.loc[i, key+'_up'] = up_value
/var/folders/mc/hhcvbs7n2jq7d_hr2d6qvjhm0000gq/T/ipykernel_3951/2331560644.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '18.6' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  table.loc[i, key+'_up'] = up_value
/var/folders/mc/hhcvbs7n2jq7d_hr2d6qvjhm0000gq/T/ipykernel_3951/2331560644.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.4' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  table.l

,Name,Class,gamma,d,vpecp,pkv,mcomp,mnoncomp,porb,d_up,d_down,vpecp_up,vpecp_down,pkv_up,pkv_down,mcomp_up,mcomp_down,mnoncomp_up,mnoncomp_down
0,2S 0921-630,NS-LMXB,44.4\pm 2.4 [1],8.5^\ast,38.2,58.9,1.4 [2],0.3 [2],9.00 [3],NaN,None,18.6,7.4,10.4,3.9,0.1,0.1,0.0,0.0
1,4U 1254-69,NS-LMXB,183.0\pm 3.0 [4],{13.0}^\dag,155.3,202.9,1.5 [4],0.5 [4],0.16 [5],3.0,3.0,29.0,17.6,81.1,52.9,0.3,0.3,0.1,0.1
2,Cen X-4,NS-LMXB,194.5\pm 0.2 [6],{1.4}^\dag,418.7,457.2,1.9 [6],0.3 [6],0.63 [6],0.3,0.3,72.1,68.5,84.1,137.8,0.4,0.8,0.2,0.1
3,Sco X-1,NS-LMXB,-113.8\pm 0.6 [7],2.2,167.5,210.6,1.4^\ast,0.4^\ast [8],0.79 [9],0.1,0.1,10.3,9.8,19.0,29.0,NaN,None,NaN,None
4,4U 1636-536,NS-LMXB,-34.0\pm 5.0 [10],{6.0}^\dag,164.7,194.6,1.4^\ast,0.5^\ast [10],0.16 [10],0.5,0.5,12.6,12.6,15.0,25.7,NaN,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Gaia BH2,BH-LMNI,-4.2\pm 0.2 [137],1.2,25.2,34.1,8.9 [137],1.1 [137],1276.70 [137],0.0,0.0,9.7,8.7,5.0,5.1,0.3,0.3,0.2,0.2
85,J15274848,NS-LMNI,-26.1\pm 0.7 [138],0.1,13.8,18.9,1.0 [138],0.6 [138],0.26 [139],0.0,0.0,7.1,3.5,3.1,3.0,0.1,0.1,0.1,0.1
86,Gaia BH1,BH-LMNI,45.5\pm 0.8 [140],0.5,76.4,71.3,9.8 [140],0.9 [140],185.59 [140],0.0,0.0,4.0,3.1,9.2,10.8,0.2,0.2,0.1,0.1
87,AS 386,BH-HMNI,-31.8\pm 2.6 [141],5.9,10.8,20.6,>7.0 [141],7.0 [141],131.27 [141],0.5,0.4,8.0,4.4,2.2,2.7,NaN,None,1.0,1.0


## Plot peculiar velocities vs masses

In [7]:
# Convert the specified columns to float type
Zhao_2023_Table3_clean[['pkv', 'pkv_up', 'pkv_down']] = Zhao_2023_Table3_clean[['pkv', 'pkv_up', 'pkv_down']].astype(float)
display(Zhao_2023_Table3_clean[['pkv', 'pkv_up', 'pkv_down']] )

def cleanup_column(data_column):
    print(type(data_column[0]))
    # Check if data is a string
    if not isinstance(data_column[0], str):
        return data_column
    
    # Split the column at '[' and take the first part
    col = data_column.str.split('[').str[0]
    
    # Apply replacements only to string values
    col = [x.replace(r'^\ast', '').replace('>', '').replace('<', '').replace(r'\nodata', '-1') if isinstance(x, str) else x for x in col]

    # Convert cleaned strings to floats, skipping non-convertible values
    col = [float(x) for x in col ]
    
    return col

    
# def cleanup_column(data_column):
#     print(type(data_column[0]) )
#     # check if data is a string
#     if not isinstance(data_column[0], str):
#         return data_column
#     col = data_column.str.split('[').str[0]
#     col = [x.replace(r'^\ast', '') for x in col]
#     col = [x.replace(r'>', '') for x in col]
#     col = [x.replace(r'<', '') for x in col]
#     col = [x.replace(r'\nodata', '-1') for x in col]
#     col = [float(x) for x in col]
#     return col


M_1 = cleanup_column(Zhao_2023_Table3_clean['mcomp'])
M_1_up = cleanup_column(Zhao_2023_Table3_clean['mcomp_up'])
M_1_down = cleanup_column(Zhao_2023_Table3_clean['mcomp_up'])

M_2 = cleanup_column(Zhao_2023_Table3_clean['mnoncomp'])
print(M_1)


# Zhao_2023_Table3_clean[['mcomp_up', 'mcomp_down']] = Zhao_2023_Table3_clean[['mcomp_up', 'mcomp_down']].astype(float)
# M_1 = Zhao_2023_Table3_clean['mcomp'].str.split('[').str[0]
# M_1 = [m1.replace(r'^\ast', '') for m1 in M_1]
# M_1 = [m1.replace(r'>', '') for m1 in M_1]
# M_1 = [m1.replace(r'<', '') for m1 in M_1]
# M_1 = [m1.replace(r'\nodata', '-1') for m1 in M_1]
# M_1 = [float(m1) for m1 in M_1]
# print(M_1)

# Zhao_2023_Table3_clean[['mnoncomp_up', 'mnoncomp_down']] = Zhao_2023_Table3_clean[['mnoncomp_up', 'mnoncomp_down']].astype(float)
# M_2 = Zhao_2023_Table3_clean['mnoncomp'].str.split('[').str[0]
# M_2 = [m2.replace(r'^\ast', '') for m2 in M_2]
# M_2 = [m2.replace(r'>', '') for m2 in M_2]
# M_2 = [m2.replace(r'<', '') for m2 in M_2]
# M_2 = [m2.replace(r'\nodata', '-1') for m2 in M_2]

# M_2 = [float(m2) for m2 in M_2]
# print(M_2)


# display(Zhao_2023_Table3_clean[['mnoncomp', 'mnoncomp_up', 'mnoncomp_down']] )

,pkv,pkv_up,pkv_down
0,58.9,10.4,3.9
1,202.9,81.1,52.9
2,457.2,84.1,137.8
3,210.6,19.0,29.0
4,194.6,15.0,25.7
...,...,...,...
84,34.1,5.0,5.1
85,18.9,3.1,3.0
86,71.3,9.2,10.8
87,20.6,2.2,2.7


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
[1.4, 1.5, 1.9, 1.4, 1.4, 1.5, 2.1, 1.4, 1.4, 1.4, 2.0, 2.0, 1.4, 1.4, 1.7, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.6, 1.4, 1.4, 2.1, 1.4, 1.6, 1.7, 1.4, 1.9, 1.4, 1.4, 1.4, 1.0, 2.0, 1.4, 3.7, 1.4, 1.4, 6.6, 7.5, 11.0, 8.9, 7.6, 9.4, 11.7, 6.0, 5.9, 6.4, 7.4, 8.4, -1.0, 12.4, 9.0, 6.2, 6.4, 4.3, 21.2, 4.1, 2.0, 1.6, 1.6, 1.4, 2.0, 1.4, 1.4, 1.8, 2.1, 2.0, 1.4, 1.4, 1.4, 2.2, 1.2, 1.8, 1.3, 1.7, 2.3, 1.6, 3.3, 1.3, 1.2, 1.4, 8.9, 1.0, 9.8, 7.0, 1.3]


In [12]:
print(M_1)
print(M_1_down)
print(M_1_up)

print(M_1 - M_1_down)

[1.4, 1.5, 1.9, 1.4, 1.4, 1.5, 2.1, 1.4, 1.4, 1.4, 2.0, 2.0, 1.4, 1.4, 1.7, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.6, 1.4, 1.4, 2.1, 1.4, 1.6, 1.7, 1.4, 1.9, 1.4, 1.4, 1.4, 1.0, 2.0, 1.4, 3.7, 1.4, 1.4, 6.6, 7.5, 11.0, 8.9, 7.6, 9.4, 11.7, 6.0, 5.9, 6.4, 7.4, 8.4, -1.0, 12.4, 9.0, 6.2, 6.4, 4.3, 21.2, 4.1, 2.0, 1.6, 1.6, 1.4, 2.0, 1.4, 1.4, 1.8, 2.1, 2.0, 1.4, 1.4, 1.4, 2.2, 1.2, 1.8, 1.3, 1.7, 2.3, 1.6, 3.3, 1.3, 1.2, 1.4, 8.9, 1.0, 9.8, 7.0, 1.3]
[0.1, 0.3, 0.4, nan, nan, 0.3, 0.9, nan, nan, nan, 0.7, 0.4, nan, nan, 0.2, nan, nan, nan, nan, nan, nan, nan, 0.6, nan, nan, 0.2, nan, 0.1, 0.3, nan, 0.6, nan, nan, nan, 0.2, 0.2, nan, 1.3, nan, nan, 0.3, 0.7, 2.1, 1.6, nan, 1.0, 3.9, 0.4, 3.6, 1.5, nan, 0.8, nan, 2.0, 0.2, 0.9, 0.6, 0.4, 2.2, 1.4, 0.0, 0.2, 0.2, nan, nan, nan, nan, 0.1, 0.1, 0.1, nan, nan, nan, 0.2, 0.3, nan, 0.3, 0.2, 0.2, 0.3, 1.1, 0.1, 0.0, 0.7, 0.3, 0.1, 0.2, nan, nan]
[0.1, 0.3, 0.4, nan, nan, 0.3, 0.9, nan, nan, nan, 0.7, 0.4, nan, nan, 0.2, nan, nan, nan, nan, nan, na

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [340]:
for i in range(len(Zhao_2023_Table3_clean)):

    pkv_err = np.array([[Zhao_2023_Table3_clean['pkv'][i] - Zhao_2023_Table3_clean['pkv_down'][i]], 
                            [Zhao_2023_Table3_clean['pkv_up'][i] - Zhao_2023_Table3_clean['pkv'][i]]])


    M_1_err = np.array([[M_1 - M_1_down], 
                            [M_1_up - M_1]])


    plt.errorbar(x=Zhao_2023_Table3_clean['pkv'][i], xerr=pkv_err, y=M_1[i], yerr=M_1_err, fmt='o')  

plt.show()



TypeError: unsupported operand type(s) for -: 'list' and 'list'